# Трекинг экспериментов в ClearML

## Интеграция ClearML в код обучения

Сначала перенесём код обучения сети, который мы создали в первом уроке темы, в формат ClearML-запуска.

Основная идея — обернуть скрипт вызовом Task.init(), который создаст запись об эксперименте на сервере. Затем с помощью методов Task.connect() и task.get_logger().report_scalar() будем передавать параметры и метрики.

Расширим наш PyTorch-код — добавим в начале скрипта строки:

```python
from clearml import Task

task = Task.init(project_name="SImpleNN Project", task_name="Experiment #1")

config = {"learning_rate": 0.01,
          "batch_size": 16,
          "epochs": 3}

task.connect(config)
```

Здесь:

- Task.init(...) создаёт новую запись эксперимента в указанном проекте. Проект — это папка в веб-интерфейсе, где будут храниться эксперименты.
- task.connect(config) отправляет словарь config на сервер, чтобы эти параметры автоматически отображались в UI.

Внутри цикла обучения будем логировать метрику (loss). Например:

```python

for epoch in range(config['epochs']):
    # ... код обучения одной эпохи, вычисляем loss ...
    loss_value = train_loop(...)  # ваша функция обучения возвращает текущий loss


    # Логируем loss в ClearML
    task.get_logger().report_scalar("Loss", "train", epoch, loss_value)
```

Метод report_scalar(name, series, step, value) отправляет одно значение метрики на сервер. Здесь:

- "Loss" — название графика,
- "train"  — подпапка (серия) на графике,
- epoch — номер шага (на оси X),
- loss_value — само значение.
- Теперь в UI появится график падения потерь по эпохам.

In [ ]:
from clearml import Task
import torch
from torch import nn

from SimpleNN import SimpleNN

# 2. Инициализация ClearML-задачи
task = Task.init(project_name="SimpleNN Project", task_name="Experiment #1")
config = {"learning_rate": 0.01, "batch_size": 4, "epochs": 100}
task.connect(config)

# 3. Создаём модель, оптимизатор и функцию потерь
model = SimpleNN(in_features=5).to(torch.device('cpu'))
optimizer = torch.optim.SGD(model.parameters(), lr=config["learning_rate"])
criterion = nn.MSELoss()

# 4. Генерируем случайные входные данные и метки
torch.manual_seed(0)
X = torch.randn(3, 5)
y = torch.randn(3, 1)

# 5. Обучение модели
for epoch in range(config["epochs"]):
    outputs = model(X)
    loss = criterion(outputs, y)


    # Шаг обучения
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Логируем loss с указанием итерации
    task.get_logger().report_scalar("Loss", "train", loss.item(), epoch)
    print(f"Epoch {epoch + 1}/{config['epochs']}, Epoch {epoch}, Loss: {loss.item():.4f}") 

ClearML Task: created new task id=c5cb858cd6a74f11bf8972425ef0fb1a
2025-10-24 20:06:18,941 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/331a211f06cb46079db4f1ed5d3193a4/experiments/c5cb858cd6a74f11bf8972425ef0fb1a/output/log
Epoch 1/100, Epoch 0, Loss: 0.1321
Epoch 2/100, Epoch 1, Loss: 0.1238
Epoch 3/100, Epoch 2, Loss: 0.1164
Epoch 4/100, Epoch 3, Loss: 0.1096
Epoch 5/100, Epoch 4, Loss: 0.1034
Epoch 6/100, Epoch 5, Loss: 0.0978
Epoch 7/100, Epoch 6, Loss: 0.0927
Epoch 8/100, Epoch 7, Loss: 0.0880
Epoch 9/100, Epoch 8, Loss: 0.0837
Epoch 10/100, Epoch 9, Loss: 0.0798
Epoch 11/100, Epoch 10, Loss: 0.0762
Epoch 12/100, Epoch 11, Loss: 0.0729
Epoch 13/100, Epoch 12, Loss: 0.0699
Epoch 14/100, Epoch 13, Loss: 0.0671
Epoch 15/100, Epoch 14, Loss: 0.0645
Epoch 16/100, Epoch 15, Loss: 0.0621
Epoch 17/100, Epoch 16, Loss: 0.0598
Epoch 18/100, Epoch 17, Loss: 0.0577
Epoch 19/100, Epoch 18, Loss: 0.0558
Epoch 20/10

### TASK STOPPED - USER ABORTED - STATUS CHANGED ###


# Обзор интерфейса ClearML

Веб-интерфейс ClearML позволяет легко наблюдать за экспериментами.

На вкладке PROJECTS показан список всех проектов, а в каждой папке хранятся все запуски вашего проекта: статус (Running, Completed), названия, теги, а также превью графиков метрик.

При выборе эксперимента открывается детальная страница: в ней есть вкладки с метриками (графики training/validation loss, accuracy и т. д.), логами запуска, списком сохранённых артефактов (моделей, чекпоинтов) и параметрами. 

Можно переключаться между разными метриками и сериями, масштабировать графики и сравнивать несколько экспериментов.

Кроме того, в UI есть другие разделы: в Model Endpoints хранятся сохранённые модели, а Datasets помогает удобно версионировать данные.

# Работа с датасетами через ClearML

ClearML умеет хранить версии датасетов. Используя сохранённые версии, при повторных запусках вы гарантированно используете те же данные.

Добавить датасет в ClearML можно двумя способами: с помощью CLI (clearml-data) или программно через класс Dataset.

Добавьте датасет в ClearML удобным вам способом.

## Через CLI
Если вы работаете со своим окружением (не на платформе), перед началом работы установите инструмент clearml-data. Для этого выполните команду в терминале:

```bash
pip install clearml-data
```

Затем добавить датасет:

```bash

clearml-data create --project "SimpleNN Project" --name "synthetic_data"
clearml-data add --files data.csv

clearml-data close
```

Эти команды создадут в проекте SimpleNN Project новый датасет synthetic_data, добавят в него файл data.csv и зафиксируют версию датасета. 

После этого в UI в разделе Datasets появится запись о созданном датасете с версией «Current».

## Через ячейку ноутбука:

```python

from clearml import Dataset


dataset = Dataset.create(dataset_project="SimpleNN Project", dataset_name="synthetic_data")


# Генерируем случайные данные как в прошлом задании
torch.manual_seed(0)
X = torch.randn(3, 5)   # форма [3, 5]
y = torch.randn(3, 1)   # форма [3, 1]


# Превращаем X и y в pandas.DataFrame
# Сначала приводим к numpy, затем создаём DataFrame

df = pd.DataFrame(
    X.numpy(),
    columns=[f"f{i+1}" for i in range(5)]   # f1, f2, f3, f4, f5
)
df["label"] = y.numpy()  # добавляем колонку "label"


df.to_csv("data.csv", index=False)
dataset.add_files("/path/to/data.csv")
dataset.upload()
dataset.finalize()
```

Код создаст новый датасет и загрузит файл на сервер. Чтобы в дальнейшем использовать датасет в коде обучения, достаточно вызвать:

```python

dataset = Dataset.get(dataset_project="SimpleNN Project", dataset_name="synthetic_data")
data_path = dataset.get_local_copy()
# Теперь считываем data.csv из папки data_path
```

In [2]:
import pandas as pd
from clearml import Dataset


dataset = Dataset.create(dataset_project="SimpleNN Project", dataset_name="synthetic_data")


# Генерируем случайные данные как в прошлом задании
torch.manual_seed(0)
X = torch.randn(3, 5)   # форма [3, 5]
y = torch.randn(3, 1)   # форма [3, 1]


# Превращаем X и y в pandas.DataFrame
# Сначала приводим к numpy, затем создаём DataFrame

df = pd.DataFrame(
    X.numpy(),
    columns=[f"f{i+1}" for i in range(5)]   # f1, f2, f3, f4, f5
)
df["label"] = y.numpy()  # добавляем колонку "label"


df.to_csv("data.csv", index=False)
dataset.add_files("data.csv")
dataset.upload()
dataset.finalize()

ClearML results page: https://app.clear.ml/projects/94d1c1f9809a47b4ab8b339b624eb987/experiments/da3188be1578477fa4ef5884434b61bf/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/94d1c1f9809a47b4ab8b339b624eb987/experiments/da3188be1578477fa4ef5884434b61bf
Uploading dataset changes (1 files compressed to 245 B) to https://files.clear.ml
File compression and upload completed: total size 245 B, 1 chunk(s) stored (average size 245 B)


True

In [1]:

from clearml import Task, Dataset

import pandas as pd

import torch

from torch import nn



# 1. Определяем класс SimpleNN
from SimpleNN import SimpleNN


# 2. Инициализация ClearML-задачи
task = Task.init(project_name="SimpleNN Project", task_name="Experiment #1")
config = {"learning_rate": 0.01, "batch_size": 2, "epochs": 20}



# Подключение к task
task.connect(config)

# 3. Создаём модель, оптимизатор и функцию потерь
model = SimpleNN(in_features=5).to(torch.device('cpu'))
optimizer = torch.optim.SGD(model.parameters(), lr=config["learning_rate"])
criterion = nn.MSELoss()


# 4. Генерируем данные
torch.manual_seed(0)
X = torch.randn(3, 5)   # форма [3, 5]
y = torch.randn(3, 1)   # форма [3, 1]


# 5. Обучение модели
for epoch in range(config["epochs"]):
    outputs = model(X)
    loss = criterion(outputs, y)


    # Шаг обучения
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    # Логируем loss через task с указанием итерации
    task.get_logger().report_scalar("Loss", "train", loss.item(), epoch)
    print(f"Epoch {epoch + 1}/{config['epochs']}, Epoch {epoch}, Loss: {loss.item():.4f}")

ClearML Task: overwriting (reusing) task id=c5cb858cd6a74f11bf8972425ef0fb1a
2025-10-24 20:31:09,559 - clearml.Task - INFO - No repository found, storing script code instead
ClearML results page: https://app.clear.ml/projects/331a211f06cb46079db4f1ed5d3193a4/experiments/c5cb858cd6a74f11bf8972425ef0fb1a/output/log
Epoch 1/20, Epoch 0, Loss: 0.1321
Epoch 2/20, Epoch 1, Loss: 0.1238
Epoch 3/20, Epoch 2, Loss: 0.1164
Epoch 4/20, Epoch 3, Loss: 0.1096
Epoch 5/20, Epoch 4, Loss: 0.1034
Epoch 6/20, Epoch 5, Loss: 0.0978
Epoch 7/20, Epoch 6, Loss: 0.0927
Epoch 8/20, Epoch 7, Loss: 0.0880
Epoch 9/20, Epoch 8, Loss: 0.0837
Epoch 10/20, Epoch 9, Loss: 0.0798
Epoch 11/20, Epoch 10, Loss: 0.0762
Epoch 12/20, Epoch 11, Loss: 0.0729
Epoch 13/20, Epoch 12, Loss: 0.0699
Epoch 14/20, Epoch 13, Loss: 0.0671
Epoch 15/20, Epoch 14, Loss: 0.0645
Epoch 16/20, Epoch 15, Loss: 0.0621
Epoch 17/20, Epoch 16, Loss: 0.0598
Epoch 18/20, Epoch 17, Loss: 0.0577
Epoch 19/20, Epoch 18, Loss: 0.0558
Epoch 20/20, Epoch 1